In [2]:
from dotenv import load_dotenv
import pandas as pd
import os
from pymongo import MongoClient

load_dotenv()
cluster_uri = os.environ.get("MONGODB_URI")
client = MongoClient(cluster_uri)
db = client["MSCI446_DB"]

### This module is for loading the data and transforming them into dataframes

In [ ]:
# This module converts the objects pulled from Mono DB into a dataframe
collection_forecast = db["Load_Forecast"]
collection_solar = db["Solar_Forecast"]
collection_wind = db["Wind_Forecast"]
collection_gas_price = db["Gas_Prices"]
collection_gen_outages = db["Gen_Outages"]
collection_historical_da_price = db["Historical_DA_Prices"]

documents_forecast = list(collection_forecast.find())
documents_solar = list(collection_solar.find())
documents_wind = list(collection_wind.find())
documents_gas_price = list(collection_gas_price.find())
documents_gen_outages = list(collection_gen_outages.find())
documents_historical_da_price = list(collection_historical_da_price.find())

# Converting documents into dataframes
df_forecast = pd.DataFrame(documents_forecast)
df_solar = pd.DataFrame(documents_solar)
df_wind = pd.DataFrame(documents_wind)
df_gas_price = pd.DataFrame(documents_gas_price)
df_gen_outages = pd.DataFrame(documents_gen_outages)
df_historical_da_price = pd.DataFrame(documents_historical_da_price)

# Dropping the id column
df_forecast.drop('_id', axis=1, inplace=True)
df_solar.drop('_id', axis=1, inplace=True)
df_wind.drop('_id', axis=1, inplace=True)
df_gas_price.drop('_id', axis=1, inplace=True)
df_gen_outages.drop('_id', axis=1, inplace=True)
df_historical_da_price.drop('_id', axis=1, inplace=True)

print(df_forecast.head())
print(df_solar.head())
print(df_wind.head())
print(df_gas_price.head())
print(df_gen_outages.head())
print(df_historical_da_price.head())

### This module is for getting unique areas from each dataframe

In [ ]:
# Get unique area values from each dataframe
unique_areas_solar = df_solar['area'].unique()
unique_areas_wind = df_wind['area'].unique()
unique_areas_load_forecast = df_forecast['forecast_area'].unique()
unique_areas_historical_da_price = df_historical_da_price['zone'].unique()

# Print the unique area values
print("Unique areas in Solar Forecast table:", unique_areas_solar)
print("Unique areas in Wind Forecast table:", unique_areas_wind)
print("Unique areas in Load Forecast:", unique_areas_load_forecast)
print("Unique areas in Historical DA Price:", unique_areas_historical_da_price)

### This module is for renaming column headings to common datetime heading

In [ ]:
# Renaming date-time columns in various dataframes to a common date-time column heading
df_forecast.rename(columns={'forecast_hour_beginning_ept': 'datetime_beginning_ept'}, inplace=True)
df_gen_outages.rename(columns={'forecast_date': 'datetime_beginning_ept'}, inplace=True)
df_gas_price.rename(columns={'Date': 'datetime_beginning_ept'}, inplace=True)

### This module is for preparing load_forecast dataset

In [ ]:
# Changing the 'datetime_beginning_ept' column into datetime format
df_forecast['datetime_beginning_ept'] = pd.to_datetime(df_forecast['datetime_beginning_ept'])

# Filtering out rows where the date is earlier than 2022
df_forcast = df_forecast[df_forecast['datetime_beginning_ept'].dt.year >= 2022]

# Set the 'datetime_beginning_ept' column as the index, resample to hourly increments, and forward fill the gas prices
df_forecast_index = df_forecast.set_index('datetime_beginning_ept').resample('H').ffill()

# Reset the index to make 'datetime_beginning_ept' a column again
df_forecast_index.reset_index(inplace=True)

# Convert the 'datetime_beginning_ept' column from datetime to string
df_forecast_index['datetime_beginning_ept'] = df_forecast_index['datetime_beginning_ept'].dt.strftime('%d/%m/%Y %H:%M:%S %p')

# Print the first few rows of the resampled DataFrame to verify the results
print(df_forecast_index.tail())

### This module is for preparing the gen_outages dataset

In [ ]:
# Converting the column into date time
df_gen_outages['datetime_beginning_ept'] = pd.to_datetime(df_gen_outages['datetime_beginning_ept'])

# The gen_outage dataframe has duplicate dates (rows) that and their values need to be averaged 
df_gen_outages = df_gen_outages.groupby('datetime_beginning_ept').mean().reset_index()

# Filtering out rows where the date is earlier than 2022
df_gen_outages = df_gen_outages[df_gen_outages['datetime_beginning_ept'].dt.year >= 2022]

# Set the 'datetime_beginning_ept' column as the index, resample to hourly increments, and forward fill the gas prices
df_gen_outages_index = df_gen_outages.set_index('datetime_beginning_ept').resample('H').ffill()

# Reset the index to make 'datetime_beginning_ept' a column again
df_gen_outages_index.reset_index(inplace = True)

# Convert the 'datetime_beginning_ept' column from datetime to string
df_gen_outages_index['datetime_beginning_ept'] = df_gen_outages_index['datetime_beginning_ept'].dt.strftime('%d/%m/%Y %H:%M:%S %p')

print(df_gen_outages_index.tail())

### This module is for preparing the gas_price dataset

In [ ]:
# Changing the 'datetime_beginning_ept' column into datetime format
df_gas_price['datetime_beginning_ept'] = pd.to_datetime(df_gas_price['datetime_beginning_ept'])

# Filtering out rows where the date is earlier than 2022
df_gas_price = df_gas_price[df_gas_price['datetime_beginning_ept'].dt.year >= 2022]

# Set the 'datetime_beginning_ept' column as the index, resample to hourly increments, and forward fill the gas prices
df_gas_price_index = df_gas_price.set_index('datetime_beginning_ept').resample('H').ffill()

# Reset the index to make 'datetime_beginning_ept' a column again
df_gas_price_index.reset_index(inplace=True)

# Convert the 'datetime_beginning_ept' column from datetime to string
df_gas_price_index['datetime_beginning_ept'] = df_gas_price_index['datetime_beginning_ept'].dt.strftime('%d/%m/%Y %H:%M:%S %p')

# Print the first few rows of the resampled DataFrame to verify the results
print(df_gas_price_index.tail())

### This module is for pivoting dataframes and changing column headings for merge

In [172]:
# Instead of dropping the duplicate rows, I just averaged them and since they're the same their value shouldn't change
wind_pivot = df_wind.pivot_table(index='datetime_beginning_ept', columns='area', values='wind_generation_mw', aggfunc='mean')
solar_pivot = df_solar.pivot_table(index='datetime_beginning_ept', columns='area', values='solar_generation_mw', aggfunc='mean')
load_forecast_pivot = df_forecast.pivot_table(index='datetime_beginning_ept', columns ='forecast_area', values = 'forecast_load_mw', aggfunc = 'mean')
historical_da_pivot = df_historical_da_price.pivot_table(index='datetime_beginning_ept', columns ='zone', values = 'total_lmp_da', aggfunc = 'mean')

# Renaming column headings to differentiate from other columns
load_forecast_pivot = load_forecast_pivot.rename(columns={col: f"{col}_forecast" for col in load_forecast_pivot.columns if col != 'datetime_beginning_ept'})
historical_da_pivot = historical_da_pivot.rename(columns={col: f"{col}_historical_da" for col in historical_da_pivot.columns if col != 'datetime_beginning_ept'})

### This module merges the dataframes into one

In [173]:
# Merge the pivoted DataFrames along the dates
merged_df = pd.merge(solar_pivot, wind_pivot, on='datetime_beginning_ept', suffixes=('_solar', '_wind'))

merged_df = pd.merge(merged_df, load_forecast_pivot, on = 'datetime_beginning_ept')

merged_df = pd.merge(merged_df, historical_da_pivot, on = 'datetime_beginning_ept')

merged_df = pd.merge(merged_df, df_gen_outages_index, on = 'datetime_beginning_ept')

merged_df = pd.merge(merged_df, df_gas_price_index, on = 'datetime_beginning_ept')

print(merged_df.tail())


    datetime_beginning_ept  MIDATL_solar  OTHER_solar  RFC_solar  RTO_solar  \
22  12/11/2023 11:00:00 AM       480.663       59.048    973.434   2766.207   
23  12/11/2023 12:00:00 PM       360.128       70.877    894.124   2526.460   
24  12/12/2023 10:00:00 AM       741.332      156.655   1955.476   4118.816   
25  12/12/2023 11:00:00 AM       799.335      142.255   1942.117   4003.176   
26  12/12/2023 12:00:00 PM       790.113      147.676   1904.437   3869.533   

    SOUTH_solar  WEST_solar  MIDATL_wind  OTHER_wind  RFC_wind  ...  \
22     1792.773     492.771      685.213       4.342  2110.830  ...   
23     1632.336     533.996      676.656       4.332  2138.779  ...   
24     2163.340    1214.144      357.270       5.085  4379.018  ...   
25     2061.059    1142.782      364.103       5.401  4316.033  ...   
26     1965.096    1114.324      498.561       4.429  4519.631  ...   

    DOM_forecast  DUQ_forecast  EKPC_forecast  MIDATL_forecast  RTO_forecast  \
22     13700.875  